In [ ]:
import configparser
import psycopg2
from time import time
from iac import create_cluster

In [ ]:
%load_ext sql

In [ ]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

DWH_DB= config.get("CLUSTER","DB_NAME")
DWH_DB_USER= config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD= config.get("CLUSTER","DB_PASSWORD")
DWH_PORT = config.get("CLUSTER","DB_PORT")

host = create_cluster()
conn_string = "postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, host, DWH_PORT, DWH_DB)

%sql $conn_string

In [ ]:
%%sql
SELECT table_id,"table" tablename,schema schemaname,tbl_rows,unsorted,sortkey1,sortkey_num,diststyle 
FROM svv_table_info
ORDER BY schemaname, tablename

In [ ]:
%%sql
SET search_path TO dist;

(SELECT 'artists' as table, 'artist_id' as column, count(distinct artist_id)
FROM artists
UNION
SELECT 'staging_songs', 'artist_id', count(distinct artist_id)
FROM staging_songs
UNION
SELECT 'songs', 'song_id', count(distinct song_id)
FROM songs
UNION
SELECT 'staging_songs', 'song_id', count(distinct song_id)
FROM staging_songs
UNION
SELECT 'users', 'user_id', count(distinct user_id)
FROM users
UNION
SELECT 'staging_events', 'user_id', count(distinct userId)
FROM staging_events
WHERE page = 'NextSong'
UNION
SELECT 'staging_events', 'start_time', count(distinct ts)
FROM staging_events
WHERE page = 'NextSong'
UNION
SELECT 'time', 'start_time', count(distinct start_time)
FROM time)
ORDER BY 2

In [ ]:
%%sql
SET search_path TO dist;

SELECT * FROM songplays
LIMIT 5;

In [ ]:
%%sql
SET search_path TO dist;

SELECT * FROM users
LIMIT 5;

In [ ]:
%%sql
SET search_path TO dist;

SELECT * FROM artists
LIMIT 5;

In [ ]:
%%sql
SET search_path TO dist;

SELECT * FROM songs
LIMIT 5;

In [ ]:
%%sql
SET search_path TO dist;

SELECT * FROM time
LIMIT 5;